In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r drive/"My Drive"/"Dutch Tweets" .

cp: cannot open 'drive/My Drive/Dutch Tweets/allDataList_chunk9.gsheet' for reading: Operation not supported
cp: cannot open 'drive/My Drive/Dutch Tweets/allDataList_chunk6.gsheet' for reading: Operation not supported


In [ ]:
# Read location chunks 
import pandas as pd 

idx = 9

allLocation_chunk = pd.read_csv("Dutch Tweets/allLocation_chunk"+str(idx)+".csv")
allLocation_chunk.shape
allLocation_chunk.head()

,Unnamed: 0,location
0,13755,Mind yuh business
1,13756,Op het circuit.
2,13757,"🇳🇱, she/her"
3,13758,Southern California
4,13759,België & Nederland


In [ ]:
import geopy.geocoders
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

locator = Nominatim(user_agent="myGeocoder")

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)


In [ ]:
# 2- - create location column
allLocation_chunk['point_info'] = allLocation_chunk['location'].apply(geocode)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('KL, IND',), **{}).
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/geopy/geocoders/base.py", line 344, in _call_geocoder
    page = requester(req, timeout=timeout, **kwargs)
  File "/usr/lib/python3.6/urllib/request.py", line 526, in open
    response = self._open(req, data)
  File "/usr/lib/python3.6/urllib/request.py", line 544, in _open
    '_open', req)
  File "/usr/lib/python3.6/urllib/request.py", line 504, in _call_chain
    result = func(*args)
  File "/usr/lib/python3.6/urllib/request.py", line 1368, in https_open
    context=self._context, check_hostname=self._check_hostname)
  File "/usr/lib/python3.6/urllib/request.py", line 1328, in do_open
    r = h.getresponse()
  File "/usr/lib/python3.6/http/client.py", line 1356, in getresponse
    response.begin()
  File "/usr/lib/python3.6/http/client.py", line 307, in begin
    version, status, reason = self._read_status()
  File "/

In [ ]:
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
allLocation_chunk['point'] = allLocation_chunk['point_info'].apply(lambda loc: tuple(loc.point) if loc else None)

allLocation_chunk.head()

,Unnamed: 0,location,point_info,point
0,13755,Mind yuh business,None,None
1,13756,Op het circuit.,"(De Hemrik, Leeuwarden, Fryslân, Nederland, 89...","(53.2014748, 5.8455015, 0.0)"
2,13757,"🇳🇱, she/her","(Her She, Poplar Avenue, The Village, Memphis,...","(35.1172406, -89.9132695, 0.0)"
3,13758,Southern California,"(University of Southern California, Flower Str...","(34.0218833, -118.28586726915088, 0.0)"
4,13759,België & Nederland,"(Gouwbergse Loop, Hoogstraten, Antwerpen, Vlaa...","(51.5017584, 4.7815452, 0.0)"


In [ ]:
none_info = allLocation_chunk[allLocation_chunk['point'].isnull()]
none_info.shape

(474, 4)

In [ ]:
allLocation_chunk.dropna(subset=['point'], inplace=True)

In [ ]:
# 4 - split point column into latitude, longitude and altitude columns
allLocation_chunk[['latitude', 'longitude', 'altitude']] = pd.DataFrame(allLocation_chunk['point'].tolist(), index=allLocation_chunk.index,)

allLocation_chunk.head(15)

,Unnamed: 0,location,point_info,point,latitude,longitude,altitude
1,13756,Op het circuit.,"(De Hemrik, Leeuwarden, Fryslân, Nederland, 89...","(53.2014748, 5.8455015, 0.0)",53.201475,5.845502,0.0
2,13757,"🇳🇱, she/her","(Her She, Poplar Avenue, The Village, Memphis,...","(35.1172406, -89.9132695, 0.0)",35.117241,-89.913269,0.0
3,13758,Southern California,"(University of Southern California, Flower Str...","(34.0218833, -118.28586726915088, 0.0)",34.021883,-118.285867,0.0
4,13759,België & Nederland,"(Gouwbergse Loop, Hoogstraten, Antwerpen, Vlaa...","(51.5017584, 4.7815452, 0.0)",51.501758,4.781545,0.0
6,13761,Bombay,"(Mumbai, Mumbai City, Maharashtra, India, (18....","(18.9387711, 72.8353355, 0.0)",18.938771,72.835335,0.0
7,13762,Mersin,"(Mersin, Akdeniz, Mersin, Akdeniz Bölgesi, 330...","(36.8117583, 34.6292679, 0.0)",36.811758,34.629268,0.0
9,13764,"Allendale, MI","(Allendale, Allendale Charter Township, Ottawa...","(42.9903205, -85.95296849268018, 0.0)",42.990321,-85.952968,0.0
10,13765,Germany/Netherlands,"(Museum Engelandvaarders, Zeereep, de Noord, N...","(52.25172635, 4.436970391340871, 0.0)",52.251726,4.436970,0.0
11,13766,Nederland rotterdam,"(Rotterdam, Zuid-Holland, Nederland, (51.92289...","(51.9228958, 4.4631727, 0.0)",51.922896,4.463173,0.0
12,13767,"Scheveningen, Den Haag","(Scheveningen, Den Haag, Zuid-Holland, Nederla...","(52.1062075, 4.2752714, 0.0)",52.106207,4.275271,0.0


In [ ]:
!cp drive/"My Drive"/"Dutch Tweets"/*.geojson .

In [ ]:
# !pip install geopandas
# !pip install geopy


In [ ]:
import geopandas as gpd

provinces = gpd.read_file('provinces.geojson')

print(provinces.shape)
provinces.head()

(12, 4)


,name,regioFacetId,level,geometry
0,Drenthe,tcm:106-353397-1024,3,"POLYGON ((6.41328 52.98552, 6.36252 53.03397, ..."
1,Flevoland,tcm:106-353410-1024,3,"POLYGON ((5.36115 52.67573, 5.37726 52.76481, ..."
2,Friesland (Fryslân),tcm:106-353417-1024,3,"POLYGON ((5.08707 53.32307, 5.10178 53.36803, ..."
3,Gelderland,tcm:106-353445-1024,3,"POLYGON ((5.60602 51.99416, 5.59200 52.00139, ..."
4,Groningen,tcm:106-353502-1024,3,"POLYGON ((6.28698 53.34138, 6.27368 53.34527, ..."


In [ ]:
from shapely.geometry import Point, Polygon

In [ ]:
def checkProvince(p1):
  #print(len(p1))
  if p1 != None and len(p1) >= 2 :
    p1 = Point(p1[1], p1[0])  
    provinces['InProvince'] = provinces['geometry'].apply(lambda x: x.contains(p1))
    selProvince = provinces[provinces['InProvince'] == True]
    #print(selProvince)
    if selProvince.shape[0] > 0:
      return selProvince['name'].values[0]
    else:
      return False
  else:
    return False

In [ ]:
allLocation_chunk['Province'] = allLocation_chunk['point'].apply(lambda x: checkProvince(x))
allLocation_chunk.head(15)

,Unnamed: 0,location,point_info,point,latitude,longitude,altitude,Province
1,13756,Op het circuit.,"(De Hemrik, Leeuwarden, Fryslân, Nederland, 89...","(53.2014748, 5.8455015, 0.0)",53.201475,5.845502,0.0,Friesland (Fryslân)
2,13757,"🇳🇱, she/her","(Her She, Poplar Avenue, The Village, Memphis,...","(35.1172406, -89.9132695, 0.0)",35.117241,-89.913269,0.0,False
3,13758,Southern California,"(University of Southern California, Flower Str...","(34.0218833, -118.28586726915088, 0.0)",34.021883,-118.285867,0.0,False
4,13759,België & Nederland,"(Gouwbergse Loop, Hoogstraten, Antwerpen, Vlaa...","(51.5017584, 4.7815452, 0.0)",51.501758,4.781545,0.0,False
6,13761,Bombay,"(Mumbai, Mumbai City, Maharashtra, India, (18....","(18.9387711, 72.8353355, 0.0)",18.938771,72.835335,0.0,False
7,13762,Mersin,"(Mersin, Akdeniz, Mersin, Akdeniz Bölgesi, 330...","(36.8117583, 34.6292679, 0.0)",36.811758,34.629268,0.0,False
9,13764,"Allendale, MI","(Allendale, Allendale Charter Township, Ottawa...","(42.9903205, -85.95296849268018, 0.0)",42.990321,-85.952968,0.0,False
10,13765,Germany/Netherlands,"(Museum Engelandvaarders, Zeereep, de Noord, N...","(52.25172635, 4.436970391340871, 0.0)",52.251726,4.436970,0.0,Zuid-Holland
11,13766,Nederland rotterdam,"(Rotterdam, Zuid-Holland, Nederland, (51.92289...","(51.9228958, 4.4631727, 0.0)",51.922896,4.463173,0.0,Zuid-Holland
12,13767,"Scheveningen, Den Haag","(Scheveningen, Den Haag, Zuid-Holland, Nederla...","(52.1062075, 4.2752714, 0.0)",52.106207,4.275271,0.0,Zuid-Holland


In [ ]:
allLocation_chunk.to_csv("allLocation_chunk"+str(idx)+".csv", index=False)
print("Geo processing completed....allLocation_chunk"+str(idx) + ".csv")

Geo processing completed....allLocation_chunk9.csv


In [ ]:
!cp allLocation_chunk9.csv drive/"My Drive"/"Dutch Tweets"/allLocation_chunk9.csv